In [27]:
import os
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())
openai_api_key=os.environ['OPENAI_API_KEY']

In [28]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [29]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/loda.txt") 
loaded_documents = loader.load()

In [30]:
loaded_documents

[Document(metadata={'source': './data/loda.txt'}, page_content="Artificial Intelligence and Machine Learning: A Comprehensive Overview\n\nIntroduction to AI\nArtificial Intelligence (AI) represents the simulation of human intelligence in machines programmed to think and learn like humans. The field encompasses everything from basic rule-based systems to complex neural networks capable of processing natural language, recognizing patterns, and making decisions.\n\nMachine Learning Fundamentals\nMachine Learning is a subset of AI that focuses on developing systems that can learn from and make decisions based on data. Key concepts include:\n1. Supervised Learning: Training models using labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through trial and error with rewards\n4. Deep Learning: Using neural networks with multiple layers\n\nNatural Language Processing\nNLP enables computers to understand, interpret, and generate human

In [31]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chucks_of_data = text_splitter.split_documents(loaded_documents)

In [32]:
chucks_of_data

[Document(metadata={'source': './data/loda.txt'}, page_content='Artificial Intelligence and Machine Learning: A Comprehensive Overview\n\nIntroduction to AI\nArtificial Intelligence (AI) represents the simulation of human intelligence in machines programmed to think and learn like humans. The field encompasses everything from basic rule-based systems to complex neural networks capable of processing natural language, recognizing patterns, and making decisions.\n\nMachine Learning Fundamentals\nMachine Learning is a subset of AI that focuses on developing systems that can learn from and make decisions based on data. Key concepts include:\n1. Supervised Learning: Training models using labeled data\n2. Unsupervised Learning: Finding patterns in unlabeled data\n3. Reinforcement Learning: Learning through trial and error with rewards\n4. Deep Learning: Using neural networks with multiple layers'),
 Document(metadata={'source': './data/loda.txt'}, page_content='Natural Language Processing\nNL

In [33]:
len(chucks_of_data)


5

In [34]:
embeddings = OpenAIEmbeddings()
vector_db = Chroma.from_documents(chucks_of_data, embeddings)


In [35]:
retriever = vector_db.as_retriever()

In [36]:
response = retriever.invoke("What is NLP?")

In [37]:
print(response)
len(response)

[Document(metadata={'source': './data/loda.txt'}, page_content='Natural Language Processing\nNLP enables computers to understand, interpret, and generate human language. Applications include:\n- Text classification\n- Sentiment analysis\n- Machine translation\n- Question answering systems\n- Chatbots and virtual assistants\n\nComputer Vision\nComputer Vision allows machines to understand and process visual information from the world. Key areas include:\n- Image classification\n- Object detection\n- Facial recognition\n- Scene understanding\n- Video analysis\n\nDeep Learning Architecture\nNeural networks form the backbone of modern AI systems:\n1. Convolutional Neural Networks (CNNs)\n- Ideal for image processing\n- Feature extraction capabilities\n- Hierarchical pattern recognition\n\n2. Recurrent Neural Networks (RNNs)\n- Processing sequential data\n- Time series analysis\n- Natural language understanding\n\n3. Transformer Models\n- Attention mechanisms\n- BERT, GPT architectures\n- S

4

In [38]:
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

## Simple Use with LCEL

In [39]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


In [40]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

In [41]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [42]:
response = rag_chain.invoke("What is NLP?")
print(response)

NLP stands for Natural Language Processing, which enables computers to understand, interpret, and generate human language. Applications of NLP include text classification, sentiment analysis, machine translation, question answering systems, chatbots, and virtual assistants.
